In [1]:
import pandas as pd
import numpy as np
from pyhive import hive
from datetime import datetime,timedelta
from datetime import timedelta
import datetime
import gc
import re
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [3]:
final = pd.read_csv('oct_data.csv')

In [13]:
final = pd.read_csv('nov_data.csv')

In [30]:
final = pd.read_csv('dec_data.csv')

In [14]:
final.dtypes

r.mobileno         int64
bre_success        int64
confirmed_flag     int64
month             object
dtype: object

In [32]:
final

,r.mobileno,bre_success,confirmed_flag,month
0,8800353758,0,0,December
1,7498722097,1,0,December
2,9622444714,1,0,December
3,7865955287,0,0,December
4,8448771301,1,0,December
...,...,...,...,...
20684,7690961235,0,0,December
20685,8177916834,0,0,December
20686,6204946738,0,0,December
20687,9365153848,0,0,December


In [33]:
final['r.mobileno'] = pd.to_numeric(final['r.mobileno'], errors = 'coerce').astype(int)

In [34]:
final.rename(columns={'r.mobileno': 'mobilePhone'}, inplace=True)


In [35]:
final.bre_success.value_counts()

0    12841
1     7848
Name: bre_success, dtype: int64

In [36]:
final.dtypes

mobilePhone        int64
bre_success        int64
confirmed_flag     int64
month             object
dtype: object

In [37]:
int_col = ['mobilePhone','applicationCreditReportId','accountHolderTypeId','createdAt','reportDate','dateClosed',
        'openDate','dateReported','accountTypeId','accountStatusId','pincode','cityId','gender',
        'employmentTypeId','monthlyIncome','Age', 'score','accountStatusName','accountTypeName','paymentHistoryProfile','highCredit','creditLimitAmount','currentBalance','amountPastDue','dateofLastPayment']
scrub_col = ['mobilePhone','panId','accountHolderTypeId','createdAt','reportDate','dateClosed',
        'openDate','dateReported','accountTypeId','accountStatusId','pincode','cityId','gender',
        'employmentTypeId','monthlyIncome','Age', 'score','accountStatusName','accountTypeName','paymentHistoryProfile','highCredit','creditLimitAmount','currentBalance','amountPastDue','dateofLastPayment']
# /pb_analytics/analytics/acc_data/Using%20Hive%20Only/2022/05May
# date_list = ['Cibil_Scrub_31July23','Cibil_Scrub_30June23']
date_int = ['12Dec']
folder = ["/Bureau","/Internal"]
pans_data = pd.DataFrame()
mob = []
for bur in date_int:
#     if((bur == '12Dec') | (bur == '11Nov') | (bur == '10Oct') | (bur == '09Sep')):
#         year = 2022
#     else:
#         year = 2023
    print(bur)
    if bur.startswith('Cibil_Scrub'):
        print('on scrub')
        for f in folder:
            print(f)
            for i in range(1,80):
                    try:
                        path = "/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/"+str(bur)+str(f)+"/cibil_pt_0"+str(i)+".parquet.gzip"
                        print(path)
                        dd = pd.read_parquet(path,columns=scrub_col).drop_duplicates()
                        dd['mobilePhone'] = pd.to_numeric(dd['mobilePhone'], errors = 'coerce')
                        
#                         dd['createdAt'] = pd.to_datetime(dd['createdAt'])
#                         dd.drop_duplicates(subset = ['mobilePhone'], inplace = True)
    #                         dd.drop_duplicates(inplace = True)
                        dd = dd[dd.mobilePhone.isin(final['mobilePhone'])]
#                         dd = pd.merge(dd, disbursal, on = 'mobilePhone', how = 'inner')
#                         dd = dd[dd['reportDate']<=dd['disbursal_date']]
                        dd = dd.loc[~dd['mobilePhone'].isin(mob)]
                        mob = mob + dd['mobilePhone'].to_list()
                        pans_data = pans_data.append(dd)
                        print(pans_data['mobilePhone'].nunique())
                        del dd
                        gc.collect()
                    except:
                        pass
    else:
        for i in range(1, 34):
            try:
                path = '/pb_analytics/analytics/acc_data/Using Hive Only/2023/'+str(bur)+'/Cibil_'+str(i)+'.parquet.gzip'
                #path = '/pb_analytics/analytics/acc_data/Using Hive Only/2022/'+str(q)+'/Cibil_'+str(i)+'.parquet.gzip'
                print(path)
                dd = pd.read_parquet(path, columns = int_col).drop_duplicates()
                dd['mobilePhone'] = pd.to_numeric(dd['mobilePhone'], errors = 'coerce')
        #                     dd['createdAt'] = pd.to_datetime(dd['createdAt'])
        #                     dd.drop_duplicates(subset = ['mobilePhone'],inplace = True)
                dd = dd[dd.mobilePhone.isin(final['mobilePhone'])]
#                 dd = pd.merge(dd, disbursal, on = 'mobilePhone', how = 'inner')
#                 dd = dd[dd['reportDate']<=dd['disbursal_date']]
                dd = dd.loc[~dd['mobilePhone'].isin(mob)]
                mob = mob + dd['mobilePhone'].to_list()
                pans_data = pans_data.append(dd)
                print(pans_data['mobilePhone'].nunique())
                del dd
                gc.collect()
            except:
                pass







12Dec
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_1.parquet.gzip
537
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_2.parquet.gzip
1060
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_3.parquet.gzip
1552
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_4.parquet.gzip
2076
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_5.parquet.gzip
2612
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_6.parquet.gzip
3202
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_7.parquet.gzip
3760
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_8.parquet.gzip
4256
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_9.parquet.gzip
4752
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_10.parquet.gzip
5118
/pb_analytics/analytics/acc_data/Using Hive Only/2023/12Dec/Cibil_11.parquet.gzip
5519
/pb_analytics/analytics/acc_data/Using Hive Onl

In [40]:
dec = pans_data
dec

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment
49123,8006659086,4b688528-9d95-5176-b1c7-c2c32ae33949,1,2023-12-01,2023-12-01,NaT,2021-02-15,2023-10-31,52,17,262305,511,MALE,2.0,8333.0,29.0,677,Active,(BLPS-SB) Business Loan â€“ Priority Sector â€...,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,",42000,-1,19861,261,2023-10-30
49124,8006659086,4b688528-9d95-5176-b1c7-c2c32ae33949,1,2023-12-01,2023-12-01,NaT,2023-10-25,2023-10-31,36,17,262305,511,MALE,2.0,8333.0,29.0,677,Active,Kisan Credit Card,"0,",98900,-1,98900,-1,None
49125,8006659086,4b688528-9d95-5176-b1c7-c2c32ae33949,1,2023-12-01,2023-12-01,NaT,2023-01-09,2023-10-31,10,17,262305,511,MALE,2.0,8333.0,29.0,677,Active,Credit Card,"0,0,0,0,0,0,0,17,0,0,",45453,50000,36693,-1,2023-10-13
49126,8006659086,4b688528-9d95-5176-b1c7-c2c32ae33949,1,2023-12-01,2023-12-01,2023-08-03,2023-04-05,2023-10-31,69,18,262305,511,MALE,2.0,8333.0,29.0,677,Closed,Short Term Personal Loan,"0,0,0,0,0,0,0,",1000,-1,0,0,2023-08-03
56689,7092341342,8266d963-a378-5205-a190-314f796e8893,1,2023-12-01,2023-12-01,2022-09-27,2022-08-26,2022-09-30,7,18,600019,420,MALE,3.0,25000.0,34.0,778,Closed,Gold Loan,"0,0,",175000,-1,0,0,2022-09-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5765434,7985146674,1ecd6326-fc17-5724-8e05-311b06653ea0,4,2023-12-14,2023-12-14,NaT,2023-04-03,2023-10-31,61,17,208027.0,557,MALE,3.0,40000.0,24.0,723,Active,Business Loan Unsecured,"0,0,0,0,0,0,0,",100000,-1,74980,-1,2023-10-05
5765435,7985146674,1ecd6326-fc17-5724-8e05-311b06653ea0,1,2023-12-14,2023-12-14,2023-04-11,2022-06-21,2023-04-30,5,18,208027.0,557,MALE,3.0,40000.0,24.0,723,Closed,Personal Loan,"0,72,52,35,27,22,0,4,4,1,0,",50000,-1,0,0,2023-04-11
5765436,7985146674,1ecd6326-fc17-5724-8e05-311b06653ea0,3,2023-12-14,2023-12-14,NaT,2021-08-24,2023-11-01,17,17,208027.0,557,MALE,3.0,40000.0,24.0,723,Active,Commercial Vehicle Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,2...",227000,-1,103268,0,2023-10-17
5768749,9103176194,3f442343-3001-56fe-85f3-0c0d19763879,1,2023-12-14,2023-12-14,NaT,2022-10-07,2023-10-31,13,17,182311.0,181,MALE,nan,0.0,21.0,748,Active,Two-Wheeler Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,",79400,-1,29073,0,None


In [42]:
nov = nov[~nov['mobilePhone'].isin(dec['mobilePhone'])]

In [39]:
dec.dtypes

mobilePhone                   int64
applicationCreditReportId    object
accountHolderTypeId          object
createdAt                    object
reportDate                   object
dateClosed                   object
openDate                     object
dateReported                 object
accountTypeId                object
accountStatusId              object
pincode                      object
cityId                       object
gender                       object
employmentTypeId             object
monthlyIncome                object
Age                          object
score                        object
accountStatusName            object
accountTypeName              object
paymentHistoryProfile        object
highCredit                   object
creditLimitAmount            object
currentBalance               object
amountPastDue                object
dateofLastPayment            object
dtype: object

In [43]:
nov

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment,panId
20912,9389703808,ba32a207-52d1-51ba-af72-c0e29cc80216,1,2023-11-01,2023-11-01,NaT,2022-12-03,2023-10-20,39,17,229001,513,MALE,3.0,166666.0,37.0,743,Active,Mudra Loans â€“ Shishu / Kishor / Tarun,"0,0,0,0,0,0,0,0,0,22,0,",190000,-1,184906,0,2023-09-18,NaN
20913,9389703808,ba32a207-52d1-51ba-af72-c0e29cc80216,3,2023-11-01,2023-11-01,2020-12-28,2018-06-15,2020-12-31,5,18,229001,513,MALE,3.0,166666.0,37.0,743,Closed,Personal Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,220,19...",25000,-1,0,0,2019-05-23,NaN
20914,9389703808,ba32a207-52d1-51ba-af72-c0e29cc80216,1,2023-11-01,2023-11-01,2019-02-15,2018-04-30,2021-03-31,6,18,229001,513,MALE,3.0,166666.0,37.0,743,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",12490,-1,0,0,2018-12-18,NaN
20915,9389703808,ba32a207-52d1-51ba-af72-c0e29cc80216,4,2023-11-01,2023-11-01,NaT,2017-07-14,2020-12-31,12,17,229001,513,MALE,3.0,166666.0,37.0,743,Active,Overdraft,"200,0,159,128,77,77,77,77,108,77,77,0,0,169,69...",400000,-1,381081,381081,2020-12-30,NaN
20916,9389703808,ba32a207-52d1-51ba-af72-c0e29cc80216,1,2023-11-01,2023-11-01,2020-10-06,2015-12-05,2021-02-28,13,18,229001,513,MALE,3.0,166666.0,37.0,743,Closed,Two-Wheeler Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",40000,-1,0,0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604826,9715460661,NaN,1,2023-11-28,2023-11-28,2022-08-13,2022-07-11,2022-09-30,7,18,607002,422.0,Male,2,30000,30.5,773.0,Closed,Gold Loan,"0,0,0",83922.0,nan,0,nan,2022-08-13,9715460661_17Nov2023
2604827,9715460661,NaN,1,2023-11-28,2023-11-28,2022-07-11,2022-06-13,2022-08-31,7,18,607002,422.0,Male,2,30000,30.5,773.0,Closed,Gold Loan,"0,0,0",195100.0,nan,0,nan,2022-07-11,9715460661_17Nov2023
2604828,9715460661,NaN,1,2023-11-28,2023-11-28,2022-03-05,2021-03-15,2022-05-31,7,18,607002,422.0,Male,2,30000,30.5,773.0,Closed,Gold Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0",30000.0,nan,0,nan,2022-03-05,9715460661_17Nov2023
2604829,9715460661,NaN,4,2023-11-28,2023-11-28,2018-08-23,2016-07-29,2019-01-31,40,18,607002,422.0,Male,2,30000,30.5,773.0,Closed,Microfinance Business Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",90000.0,nan,0,nan,2018-08-23,9715460661_17Nov2023


In [45]:
trad_oct_dec = oct.append(nov).append(dec)

In [46]:
trad_oct_dec

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment,panId
1645,7082425403,0b4f418c-b3ff-5244-8551-e9a52f401407,1,2023-10-01,2023-10-01,NaT,2023-08-18,2023-08-31,6,17,125052,145,MALE,3.0,166666.0,25.0,639,Active,Consumer Loan,"0,",18999,-1,14249,0,None,NaN
1646,7082425403,0b4f418c-b3ff-5244-8551-e9a52f401407,1,2023-10-01,2023-10-01,NaT,2023-07-31,2023-09-19,12,17,125052,145,MALE,3.0,166666.0,25.0,639,Active,Overdraft,"0,31,0,",647000,-1,646840,-1,2023-09-19,NaN
1647,7082425403,0b4f418c-b3ff-5244-8551-e9a52f401407,1,2023-10-01,2023-10-01,NaT,2023-08-18,2023-08-31,6,17,125052,145,MALE,3.0,166666.0,25.0,639,Active,Consumer Loan,"0,",41000,-1,41000,0,None,NaN
6921,7722802424,29c398aa-2cd5-5cb0-9340-e31241a21be9,1,2023-10-01,2023-10-01,2021-10-26,2021-06-16,2021-10-31,7,18,482001,257,MALE,2.0,25000.0,36.0,767,Closed,Gold Loan,"0,0,",46831,-1,0,0,2021-10-26,NaN
6922,7722802424,29c398aa-2cd5-5cb0-9340-e31241a21be9,1,2023-10-01,2023-10-01,2020-02-09,2019-03-18,2021-03-31,6,18,482001,257,MALE,2.0,25000.0,36.0,767,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",16990,-1,0,0,2019-11-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5765434,7985146674,1ecd6326-fc17-5724-8e05-311b06653ea0,4,2023-12-14,2023-12-14,NaT,2023-04-03,2023-10-31,61,17,208027.0,557,MALE,3.0,40000.0,24.0,723,Active,Business Loan Unsecured,"0,0,0,0,0,0,0,",100000,-1,74980,-1,2023-10-05,NaN
5765435,7985146674,1ecd6326-fc17-5724-8e05-311b06653ea0,1,2023-12-14,2023-12-14,2023-04-11,2022-06-21,2023-04-30,5,18,208027.0,557,MALE,3.0,40000.0,24.0,723,Closed,Personal Loan,"0,72,52,35,27,22,0,4,4,1,0,",50000,-1,0,0,2023-04-11,NaN
5765436,7985146674,1ecd6326-fc17-5724-8e05-311b06653ea0,3,2023-12-14,2023-12-14,NaT,2021-08-24,2023-11-01,17,17,208027.0,557,MALE,3.0,40000.0,24.0,723,Active,Commercial Vehicle Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,2...",227000,-1,103268,0,2023-10-17,NaN
5768749,9103176194,3f442343-3001-56fe-85f3-0c0d19763879,1,2023-12-14,2023-12-14,NaT,2022-10-07,2023-10-31,13,17,182311.0,181,MALE,nan,0.0,21.0,748,Active,Two-Wheeler Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,",79400,-1,29073,0,None,NaN


In [47]:
oct.shape[0] + nov.shape[0] + dec.shape[0]

790336

In [51]:
trad_oct_dec.employmentTypeId = trad_oct_dec.employmentTypeId.astype(str)

In [52]:

trad_oct_dec.to_parquet('cibil_base_oct_dec.parquet.gzip', index =False, compression = 'gzip')


In [212]:
total = total[['mobilePhone', 'score']]
total.drop_duplicates(subset = 'mobilePhone', keep = 'first', inplace = True)
total

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mobilePhone,score
761,8179467226,648
1186,7877333773,693
4380,7092060007,695
4507,9896956839,580
5656,9051651247,675
...,...,...
542016,8850939643,560.0
637291,8919977616,667.0
646077,7091855954,604.0
4808042,7397379073,645.0


In [213]:
check = final[final.phone.isin(total['mobilePhone'])]
check.paid.value_counts(normalize = True)

0    0.979284
1    0.020716
Name: paid, dtype: float64

In [217]:

total['score'] = pd.to_numeric(total['score'], errors = "coerce")
aa = total.groupby('mobilePhone').agg(score = ('score','max')).reset_index()

In [201]:
check = pd.merge(check, total, left_on = 'phone', right_on = 'mobilePhone')
check['score'] = pd.to_numeric(check['score'], errors = "coerce").fillna(0).astype(int)
check.dtypes

phone           int64
Month          object
paid            int64
mobilePhone     int64
score           int64
dtype: object

In [219]:
aa.bucket = pd.cut(aa.score, bins=[-1,700,750,900],labels = ['below 700', '700-750','750+'])


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [220]:
aa.bucket.value_counts(normalize = True)

below 700    0.873850
700-750      0.081832
750+         0.044318
Name: score, dtype: float64

In [222]:
final.to_parquet('target_file.parquet.gzip', compression = 'gzip', index = False)

In [224]:
total.applicationCreditReportId = np.where(total.applicationCreditReportId.isna(),total.panId,total.applicationCreditReportId)

In [231]:
aug.applicationCreditReportId = np.where(aug.applicationCreditReportId.isna(),aug.panId,aug.applicationCreditReportId)

In [235]:
appid = total[['applicationCreditReportId']]
appid.drop_duplicates(inplace= True)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [229]:
total

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment,panId
761,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-05-08,2021-04-25,2021-05-31,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,0,",8374,-1,0,0,2021-05-08,NaN
762,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-03-31,2020-03-01,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",1000,-1,0,-1,2020-03-31,NaN
763,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-02-13,2020-11-16,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",1000,-1,0,-1,2021-02-13,NaN
764,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-31,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",1000,-1,0,0,2020-07-16,NaN
765,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-28,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",500,-1,0,0,2020-07-16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808081,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2015-01-29,2014-05-24,2021-03-31,6,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",15000.0,nan,0,nan,2014-12-05,7397379073_19May2023
4808082,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2014-10-11,2013-12-18,2021-03-31,6,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",30000.0,nan,0,nan,2014-09-02,7397379073_19May2023
4808083,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2017-08-04,2007-11-12,2017-08-31,15,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,(LABD) Current Loan Against Bank Deposits,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",14000.0,nan,0,nan,NaT,7397379073_19May2023
4808084,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2014-04-30,NaT,2014-04-30,7,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Gold Loan,0,nan,nan,0,nan,NaT,7397379073_19May2023


In [240]:
appid['tag'] = np.where(appid['applicationCreditReportId'].isin(aug['applicationCreditReportId']), 'Aug', 'Jul')

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [244]:
appid['tag'].value_counts()

Aug    158004
Jul     25105
Name: tag, dtype: int64

In [245]:
appid.to_parquet('enquiries.parquet.gzip',compression = 'gzip', index = False)

In [255]:
int_col = ['applicationCreditReportId','enquiryDate','enquiryType','amount','bureauDate']
scrub_col = ['applicationCreditReportId','enquiryDate','enquiryType','amount','bureauDate']
# /pb_analytics/analytics/acc_data/Using%20Hive%20Only/2022/05May
# date_list = ['Cibil_Scrub_31July23','Cibil_Scrub_30June23']
date_int = ['07Jul','Cibil_Scrub_31July23','06Jun','Cibil_Scrub_30June23','05May','Cibil_Scrub_31May23']
folder = ["/Bureau","/Internal"]
pans_data = pd.DataFrame()
mob = []
for bur in date_int:
#     if((bur == '12Dec') | (bur == '11Nov') | (bur == '10Oct') | (bur == '09Sep')):
#         year = 2022
#     else:
#         year = 2023
    print(bur)
    if bur.startswith('Cibil_Scrub'):
        print('on scrub')
        for f in folder:
            print(f)
            for i in range(1,80):
                try:
                    path = "/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/"+str(bur)+str(f)+"/enquiry_0"+str(i)+".parquet.gzip"
                    print(path)
                    dd = pd.read_parquet(path,columns=scrub_col).drop_duplicates()
                    dd = dd[dd.applicationCreditReportId.isin(appid['applicationCreditReportId'])]
                    dd = dd.loc[~dd['applicationCreditReportId'].isin(mob)]
                    mob = mob + dd['applicationCreditReportId'].to_list()
                    pans_data = pans_data.append(dd)
                    print(pans_data['applicationCreditReportId'].nunique())
                    del dd
                    gc.collect()
                except:
                    pass
    else:
        for i in range(1, 34):
            try:
                path = '/pb_analytics/analytics/acc_data/Using Hive Only/2023/'+str(bur)+'/enq_cbl_'+str(i)+'.parquet.gzip'
                #path = '/pb_analytics/analytics/acc_data/Using Hive Only/2022/'+str(q)+'/Cibil_'+str(i)+'.parquet.gzip'
                print(path)
                dd = pd.read_parquet(path, columns = int_col).drop_duplicates()
#                 dd['mobilePhone'] = pd.to_numeric(dd['mobilePhone'], errors = 'coerce')
        #                     dd['createdAt'] = pd.to_datetime(dd['createdAt'])
        #                     dd.drop_duplicates(subset = ['mobilePhone'],inplace = True)
                dd = dd[dd.applicationCreditReportId.isin(appid['applicationCreditReportId'])]
#                 dd = pd.merge(dd, disbursal, on = 'mobilePhone', how = 'inner')
#                 dd = dd[dd['reportDate']<=dd['disbursal_date']]
                dd = dd.loc[~dd['applicationCreditReportId'].isin(mob)]
                mob = mob + dd['applicationCreditReportId'].to_list()
                pans_data = pans_data.append(dd)
                print(pans_data['applicationCreditReportId'].nunique())
                del dd
                gc.collect()
            except:
                pass







07Jul
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_1.parquet.gzip
6539
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_2.parquet.gzip
11582
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_3.parquet.gzip
13700
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_4.parquet.gzip
17232
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_5.parquet.gzip
22972
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_6.parquet.gzip
29636
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_7.parquet.gzip
35624
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_8.parquet.gzip
39976
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_9.parquet.gzip
44333
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_10.parquet.gzip
49928
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_11.parquet.gzip
54768
/pb_analytics/

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_059.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_060.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_061.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_062.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_063.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_064.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_065.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_066.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau

139011
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_2.parquet.gzip
139121
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_3.parquet.gzip
139209
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_4.parquet.gzip
139261
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_5.parquet.gzip
139331
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_6.parquet.gzip
139473
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_7.parquet.gzip
139624
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_8.parquet.gzip
139883
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_9.parquet.gzip
140328
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_10.parquet.gzip
140674
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_11.parquet.gzip
141182
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_12.parquet.gzip
141721


/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_052.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_053.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_054.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_055.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_056.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_057.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_058.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_059.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_047.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_048.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_049.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_050.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_051.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_052.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_053.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_054.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub

156636
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_013.parquet.gzip
156681
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_014.parquet.gzip
156689
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_015.parquet.gzip
156694
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_016.parquet.gzip
156737
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_017.parquet.gzip
156744
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_018.parquet.gzip
156746
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_019.parquet.gzip
156751
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_020.parquet.gzip
156807
/pb_analytics/analytics/240/Sarth

157348
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_05.parquet.gzip
157349
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_06.parquet.gzip
157351
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_07.parquet.gzip
157356
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_08.parquet.gzip
157356
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_09.parquet.gzip
157356
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_010.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_011.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_012.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_

In [268]:
pans_data['bureauDate'] = pd.to_datetime(pans_data['enquiryDate'])
pans_data

,applicationCreditReportId,enquiryDate,enquiryType,amount,bureauDate
13,fd56f252-7d13-5c85-b36f-2f514763738a,2022-08-04,Personal Loan,95000,2022-08-04
166,fd56f252-7d13-5c85-b36f-2f514763738a,2023-01-03,Consumer Loan,4000,2023-01-03
450,3a942953-803d-5c6a-be32-6731765432e6,2023-02-18,Personal Loan,3000,2023-02-18
497,3a942953-803d-5c6a-be32-6731765432e6,2022-04-13,Personal Loan,5000,2022-04-13
519,3a942953-803d-5c6a-be32-6731765432e6,2022-05-05,Two-Wheeler Loan,86000,2022-05-05
...,...,...,...,...,...
5905259,8264826452_19May2023,2014-10-09 00:00:00,Gold Loan,50000,2014-10-09
5905260,8264826452_19May2023,2014-05-17 00:00:00,Two-Wheeler Loan,47000,2014-05-17
5905261,8264826452_19May2023,2012-11-07 00:00:00,Consumer Loan,25000,2012-11-07
5905262,8264826452_19May2023,2011-06-15 00:00:00,Two-Wheeler Loan,43000,2011-06-15


In [271]:
pans_data['amount'] = pans_data['amount'].astype(str)

In [272]:

pans_data.to_parquet('enquiriesdump.parquet.gzip', compression = 'gzip', index = False)